In [18]:
#هعمل استدعاء للمكتبات بتاعتي 
#nltk هي platform لبايثون تستخدم ل اللغات البشريه الطبيعيه 
#و هي لغه لمعالجه النصوص 
import nltk
#Lemmatizer هي اداه لجمع الاشكال المختلفه لكلمه واحده معا 
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
#JSON (JavaScript Object Notation)
# صيغه لتخزين و تجميع البيانات و هي معموله ب الجافا اسكريبت 
import json
''''pickleثنائية لغرض سَلسلَة وإلغاء سَلسَلَة بنية كائنات بايثون
و تطلق ايضا علي على العملية التي يتحوّل فيها تسلسل هرمي لكائن بايثون إلى تدفق بايتات byte stream'''
import pickle
#numpy هي مكتبة يتم استخدامها فى البايثون
#وهى مكتبة يتم استعمالها لانها تحتوى على mathimatics functions
import numpy as np
# بنستخدم الموديول ده لما نكون عايزين ندخل one inpot one outpot 
from keras.models import Sequential
#layers الي بيظبط api
#Dense  Just your regular densely-connected NN layer.
# ده الي بينشط فانكشن Activation عشان تطلع outpot 
#Applies Dropout to the input. Dropout
from keras.layers import Dense, Activation, Dropout
# ده الي بيحسن gradient descent بتاعي لافضل نتيجه 
from keras.optimizers import SGD
#random ده بنستخدمه لما نكون عايزين نولد حاجه عشوائيه 
import random
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ahmed\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ahmed\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [19]:
#عندي 3 متغيرات و سبنا جواهم فاضي عشان هنعمل جواهم initialize
words=[]
classes = []
documents = []
#و قلتله تجاهل المتغيرات دي 
ignore_words = ['?', '!']
#data=pd.read_json(r"D:\PYTHON\Chatbot\intents.json")
#قرأنا فايل الجيسون 
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [20]:
# هنجهز الداتا نسحبهم من ملف الجيسون و نبدا نشتغل عليهم .
# intents و نقدر منه نجيب الاترن بنعمل لوب جوا ا
# ده عشان احول فايل الجيسون ل تكست اقدر اتعامل معاه
#بحيث انه ياخد كل جمله و يصنفها تبع الكلاس بتاعها 
# و يشوف الرد عليها ايه 
for intent in intents['intents']:
    for pattern in intent['patterns']:

#هنجيب التوكينز بتاعه كل كلمه 
        w = nltk.word_tokenize(pattern)
        words.extend(w)
#هنضيف المستند في التاج 
        documents.append((w, intent['tag']))

        # و نضيفه في الكلاس بتاعنا 
        #كده الكلاس بقي عباره عن حاجتين 
        #و الجيسون فايل و كل جلمه  موجوده في الكلاس
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [21]:
# بنشيل الكلمات المتكرره و بيرجع الكلمه لاصلها من غير اضافات و بيشيل الكلمات الاجنور الي انا قايل عليها فوق 
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# بيفرز الكلاس عندنا 
classes = sorted(list(set(classes)))
# بيعمل دكيومنت و بيحط فيه ااسئله و الاجوبه بتاعتي 
print (len(documents), "documents")
print (len(classes), "classes", classes)
# و بيخزن كل الكلمات عندي
print (len(words), "unique lemmatized words", words)

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [22]:
# بعمل تفريغ لحاجتين 
#الكلمات و بحطها في فايل بيكل للكلمات 
#و الكلاسس 
#كده عندي ملفين واحد للكلمات و واحد للكلاسس
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [24]:
# نبدا بقا نعمل تدريب و تست للداتا عندي 
training = []
# هنعمل مصفوفه فاضيه للoutpot
output_empty = [0] * len(classes)


In [25]:
#احنا دلوقتيي بنعمل حاجه ان احنا بنحول الداتا الي موجوده الي ارقام 
#عشان نقدر ندرب الموديول عليها 
for doc in documents:
    # عرفنا متغير و سبناه فاضي هيتملي من اللوب تحت 
    bag = []
    # لسته كلمات الباترن 
    pattern_words = doc[0]
    # بنحاول نجيب كل الكلام الي بنفس المعني 
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # بنملي المتغير الي فوق عن طريق array لو الكلمه تطابقت مع الباترن
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    #outpot عندي هيكون صفر للكلاس و 1 للباترن
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

In [26]:
# بنبدا نحول الداتا عندي لمصفوفه عشان ندربه عليها 
#بناخد اللاجابه random 
random.shuffle(training)
training = np.array(training)
# بحول الداتا عندي train , test 
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [27]:
# هنعمل موديل مكون من  الطبقة الأولى 128 خلية عصبية ، والطبقة الثانية 64 خلية عصبية وطبقة الإخراج الثالثة تحتوي على عدد من الخلايا العصبية
# في الاخر الي هيظهر من 9 كلاسات 8 اصفار و 1 واحد ده الكلاس بتاعه 
# Dense ده عشان نحل مشكله overfitting
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


In [28]:
# بنبني الموديل بتاعنا بديله 
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [29]:
#بندرب و بنحفظ الموديل بتاعنال
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
#خزنت الموديل في ملف h5 عشان واجه المستخدم تشغل الموديل علي طول 
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
47/47 [==============================] - 1s 11ms/step - loss: 2.2478 - accuracy: 0.0851
Epoch 2/200
47/47 [==============================] - 0s 276us/step - loss: 2.1108 - accuracy: 0.2766
Epoch 3/200
47/47 [==============================] - 0s 361us/step - loss: 2.0557 - accuracy: 0.2766
Epoch 4/200
47/47 [==============================] - 0s 361us/step - loss: 1.9116 - accuracy: 0.4255
Epoch 5/200
47/47 [==============================] - 0s 340us/step - loss: 1.8975 - accuracy: 0.3191
Epoch 6/200
47/47 [==============================] - 0s 318us/step - loss: 1.6904 - accuracy: 0.4894
Epoch 7/200
47/47 [==============================] - 0s 297us/step - loss: 1.7041 - accuracy: 0.5745
Epoch 8/200
47/47 [==============================] - 0s 340us/step - loss: 1.4462 - accuracy: 0.6383
Epoch 9/200
47/47 [==============================] - 0s 319us/step - loss: 1.3540 - accuracy: 0.6596
Epoch 10/200
47/47 [==============================] - 0s 319us/step - loss: 1.2943 - accurac

In [30]:
#هنبدا نعمل gui بتاعي 
#هنبدا نتدعي الملفات و المكتبات بتاعتي 
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
#الملف ده chatbot_model.h5 ده الي جهزناه في الخطوه الي فاتت 
model = load_model('chatbot_model.h5')
import json
import random
#و سحبنا داته الجيسون 
#و جبنا داتا الكلاس و الورد من مكتبه البيكل الي في المره السابقه 
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [31]:
# اول فانكشن معانا هي فانكشن بتعمل بتنضف الجمل 
#اول حااجه بتعملها بتحول الجمل لكلمات 
#و بتعملها زي التصيف كل الكلمات الي بمعني واحد مع بعض 
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [32]:
#بيحول الكلمات الي 0-1 و يشوفها في اي باترن 
#و برجعها علي شكل مصفوفه

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [33]:
# بدي للفانكشن الجمله و الموديل الي انا بنيته و هو بيعملي توقع 
# يجيب لكل احتمالاتها و باخد من الفانكشن الي قبلها و 
def predict_class(sentence, model):
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [35]:
#فانكشن رد الفعل 
#الفانكشن دي وظيفتها اني اديها السؤال و تجيب الرد 
#بيلف علي كل الكلاسات الي موجوده باللوب و لو لقي الكلاس الي بيدور عليه 
#بياخد اجابه عشوائيه من الاجابات  
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result


In [36]:
#بوديله رساله اول حاجه بعملها اني بيتوقع الرساله علي حسب الموديل بتاعي 
#تاني حاجه انه بيجيب الرد من الفانكشن الي قبلها 
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [37]:
#البرنامج ده زي البرنامج الي قبله بس المرادي بدأنا نستخدم مكنبه 
#tkinter المخصصه لانشاء وجهات المستخدم 
#هي واجهة المستخدم الرسومية المعيارية الفعلية لبيثون
import tkinter
from tkinter import *

In [38]:
#هنا الرساله بااخدها و بجيب الرد بتاعها 
#بعرف هنا send بتاع الرسايل عشان يبدا يعرف الاجابه 
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
    
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
            
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)


In [39]:
#هنا ببدا اجهز ال gui بتاعي من حيث الاسم و المقاسات و 
#وبقوله انه حجم واحد مش بيكبر و يصغر 
base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)


''

In [40]:
#هنا بعمل الشكل الاساسي بتاعي
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)
ChatLog.config(state=DISABLED)


In [41]:
#هنا بعمل الاسكورل الي بيطلع الصفحه فوق و تحت 
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

In [42]:
#هنا بعمل زرار الي ايمه send
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

In [43]:
#هنا بعمل البوكس الي بدخل فيه الكلام 
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")


In [44]:
#هنا بدأت ادمج الحجات الي انا عملتها فوق مع بعض عشان اطلع الشكل النهائي بتاعي
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()
